In [1]:
## Installing Library
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
raw_dataset=load_dataset('squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [5]:
## Sample Example
raw_dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [6]:
## Checking if Answer didn't have multiple answers
raw_dataset['train'].filter(lambda x:len(x['answers']['text'])!=1)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [7]:
## Now import Tokenizer so we convert sample into '[CLS] Question [SEP] Context [SEP]' Format to Feed the model.
from transformers import AutoTokenizer
model_checkpoint='bert-base-cased'

tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
## Check if Tokenizer is Fast or not ?
tokenizer.is_fast

True

In [9]:
question=raw_dataset['train'][0]['question']
context=raw_dataset['train'][0]['context']

inputs=tokenizer(question,context)
inputs

{'input_ids': [101, 1706, 2292, 1225, 1103, 6567, 2090, 9273, 2845, 1107, 8109, 1107, 10111, 20500, 1699, 136, 102, 22182, 1193, 117, 1103, 1278, 1144, 170, 2336, 1959, 119, 1335, 4184, 1103, 4304, 4334, 112, 188, 2284, 10945, 1110, 170, 5404, 5921, 1104, 1103, 6567, 2090, 119, 13301, 1107, 1524, 1104, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 171, 17506, 9538, 1110, 1103, 144, 10595, 2430, 117, 170, 14789, 1282, 1104, 8070, 1105, 9284, 119, 1135, 1110, 170, 16498, 1104, 1103, 176, 10595, 2430, 1120, 10111, 20500, 117, 1699, 1187, 1103, 6567, 2090, 25153, 1193, 1691, 1106, 2216, 17666, 6397, 3786, 1573, 25422, 13149, 1107, 8109, 119, 1335, 1103, 1322, 1104, 1103, 1514, 2797, 113, 1105, 1107, 170, 2904, 1413, 1115, 8200, 1194, 124, 11739, 1105, 11

In [10]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS]'

In [11]:
inputs=tokenizer(
    question,
    context,
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [12]:
## If context is too large that we create another feature from this featue.
inputs=tokenizer(
    raw_dataset['train'][2:6]['question'],
    raw_dataset['train'][2:6]['context'],
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [13]:
## So We 4 feature give us 19 feature
len(inputs['input_ids'])

19

In [14]:
answers=raw_dataset['train'][2:6]['answers']
start_positions = []
end_positions = []

for i,offset in enumerate(inputs['offset_mapping']):
  sample_idx=inputs['overflow_to_sample_mapping'][i]
  answer=answers[sample_idx]
  start_char=answer['answer_start'][0]
  end_char=start_char + len(answer['text'][0])
  sequence_ids=inputs.sequence_ids(i)

  ## GETTING  CONTEXT START & END
  idx=0
  while sequence_ids[idx] !=1:
    idx+=1
  context_start=idx

  while sequence_ids[idx] == 1:
    idx+=1
  context_end=idx-1

  idx=0

  if offset[context_start][0]>start_char or offset[context_end][1] < end_char:
    start_positions.append(0)
    end_positions.append(0)
  else:
    idx=context_start
    while idx <= context_end and offset[idx][0]<=start_char:
      idx+=1
    start_positions.append(idx-1)

    idx=context_end
    while idx >= context_start and offset[idx][1]>=end_char:
      idx-=1
    end_positions.append(idx+1)

start_positions,end_positions

([83, 51, 19, 0, 0, 64, 27, 0, 34, 0, 0, 0, 67, 34, 0, 0, 0, 0, 0],
 [85, 53, 21, 0, 0, 70, 33, 0, 40, 0, 0, 0, 68, 35, 0, 0, 0, 0, 0])

In [15]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: the Main Building, labels give: the Main Building


In [16]:
max_length=384
stride=128

def preprocess_training_examples(examples):
  inputs=tokenizer(
    examples['question'],
    examples['context'],
    max_length=max_length,
    truncation='only_second',
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
    )

  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  answers=examples['answers']
  start_positions = []
  end_positions = []

  for i,offset in enumerate(offset_mapping):
    sample_idx=sample_map[i]
    answer=answers[sample_idx]
    start_char=answer['answer_start'][0]
    end_char=start_char + len(answer['text'][0])
    sequence_ids=inputs.sequence_ids(i)

  ## GETTING  CONTEXT START & END
    idx=0
    while sequence_ids[idx] !=1:
      idx+=1
    context_start=idx

    while sequence_ids[idx] == 1:
      idx+=1
    context_end=idx-1

    idx=0
    if offset[context_start][0]>start_char or offset[context_end][1] < end_char:
      start_positions.append(0)
      end_positions.append(0)
    else:

      idx=context_start
      while idx <= context_end and offset[idx][0]<=start_char:
        idx+=1
      start_positions.append(idx-1)

      idx=context_end
      while idx >= context_start and offset[idx][1]>=end_char:
        idx-=1
      end_positions.append(idx+1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  return inputs

In [17]:
train_dataset=raw_dataset['train'].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_dataset["train"].column_names)
len(raw_dataset["train"]), len(train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 88729)

In [18]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 88729
})

In [19]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [20]:
validation_dataset = raw_dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_dataset["validation"].column_names,
)
len(raw_dataset["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10822)

In [21]:
validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 10822
})

In [22]:
validation_dataset[0]['example_id']

'56be4db0acb8001400a502ec'

In [23]:
small_eval_set = raw_dataset["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_dataset["validation"].column_names,
)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [24]:
len(eval_set['input_ids'])

100

In [26]:
import tensorflow as tf
from transformers import TFAutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("numpy")

batch = {k: eval_set_for_model[k] for k in eval_set_for_model.column_names}
trained_model = TFAutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

outputs = trained_model(**batch)

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [27]:
start_logits = outputs.start_logits.numpy()
end_logits = outputs.end_logits.numpy()

In [28]:
len(start_logits)

100

In [29]:
import collections

In [30]:
example_feature=collections.defaultdict(list)
example_feature

defaultdict(list, {})

In [31]:
for idx, feature in enumerate(eval_set):
    example_feature[feature["example_id"]].append(idx)

In [32]:
import numpy as np

n_best=20
max_answer_length=30
predicted_answers=[]

for example in small_eval_set:
  example_id = example["id"]
  context = example["context"]
  answers = []

  for feature_index in example_feature[example_id]:
    start_logit=start_logits[feature_index]
    end_logit=end_logits[feature_index]
    offsets=eval_set["offset_mapping"][feature_index]
    start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
    end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
    for start_index in start_indexes:
      for end_index in end_indexes:
        if offsets[start_index] is None or offsets[end_index] is None:
          continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
          continue
        answers.append({'text':context[offsets[start_index][0]:offsets[end_index][1]],'logit_score':start_logit[start_index] + end_logit[end_index]})

  best_answer = max(answers, key=lambda x: x["logit_score"])
  predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [33]:
import evaluate

metric = evaluate.load("squad")

In [34]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [35]:
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'}
{'id': '56be4db0acb8001400a502ec', 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}}


In [36]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [37]:
from tqdm.auto import tqdm
n_best=20
max_answer_length=30
predicted_answers=[]
def compute_metrics(start_logits,end_logits,features,examples):

  example_feature=collections.defaultdict(list)
  for idx, feature in enumerate(features):
    example_feature[feature["example_id"]].append(idx)

  for example in tqdm(examples):
    example_id = example["id"]
    context = example["context"]
    answers = []
    for feature_index in example_feature[example_id]:
      start_logit=start_logits[feature_index]
      end_logit=end_logits[feature_index]
      offsets=eval_set["offset_mapping"][feature_index]
      start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
      end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
      for start_index in start_indexes:
        for end_index in end_indexes:
          if offsets[start_index] is None or offsets[end_index] is None:
            continue
          if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
          answers.append({'text':context[offsets[start_index][0]:offsets[end_index][1]],'logit_score':start_logit[start_index] + end_logit[end_index]})

    if len(answers) > 0:
      best_answer = max(answers, key=lambda x: x["logit_score"])
      predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})
    else:
      predicted_answers.append({"id": example_id, "prediction_text": ""})
  theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
  return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [38]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 83.0, 'f1': 88.25000000000004}

Model Training

In [39]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [41]:
tf_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_eval_dataset = model.prepare_tf_dataset(
    validation_dataset,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [42]:
from transformers import create_optimizer
import tensorflow as tf

num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
model.fit(tf_train_dataset, epochs=num_train_epochs)

In [ ]:
predictions = model.predict(tf_eval_dataset)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    raw_dataset["validation"],
)

In [ ]:
model.save_pretrained("output_dir")


In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model=model)

context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"

question_answerer(question=question, context=context)